In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from flask import Flask, request, jsonify

In [ ]:



import pandas as pd

# Load datasets
try:
    books = pd.read_csv(r"C:\Users\abhir\Desktop\Big_data_project\Books.csv", encoding="ISO-8859-1")
    ratings = pd.read_csv(r"C:\Users\abhir\Desktop\Big_data_project\Ratings.csv", encoding="ISO-8859-1")
except Exception as e:
    print("Error loading files:", e)
    exit()

# Display the first few rows of each dataset to verify
print("Books Dataset:")
print(books.head())

print("Ratings Dataset:")
print(ratings.head())

# Merge books and ratings datasets on the 'ISBN' column
merged_data = pd.merge(ratings, books, on='ISBN')
print("\nMerged Dataset:")
print(merged_data.head())

# Filter to only include users who have rated at least 50 books
user_rating_count = merged_data['User-ID'].value_counts()
active_users = user_rating_count[user_rating_count >= 50].index
filtered_data = merged_data[merged_data['User-ID'].isin(active_users)]

# Create a pivot table for user-item matrix
user_item_matrix = filtered_data.pivot_table(index='User-ID', columns='Book-Title', values='Book-Rating')

# Fill NaN values with 0 (optional)
user_item_matrix.fillna(0, inplace=True)

# Display the user-item matrix
print("\nUser-Item Matrix:")
print(user_item_matrix)

# Compute similarity between books using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

book_similarity = cosine_similarity(user_item_matrix.T)
book_similarity_df = pd.DataFrame(book_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

# Function to get book recommendations
def get_recommendations(book_title, n_recommendations=5):
    if book_title not in book_similarity_df:
        print("Book not found in the dataset.")
        return []
    similar_books = book_similarity_df[book_title].sort_values(ascending=False).iloc[1:n_recommendations+1]
    return similar_books.index.tolist()

# Example: Get recommendations for a specific book
book_to_recommend = "The Catcher in the Rye"
recommendations = get_recommendations(book_to_recommend)

if recommendations:
    print(f"\nBooks similar to '{book_to_recommend}':")
    for idx, rec_book in enumerate(recommendations, start=1):
        print(f"{idx}. {rec_book}")

